 <h1 style="text-align: center;"> Optimized Compiler for IONQ Systems </h1>

## Setup Quantum Computer

In [1]:
from braket.devices import LocalSimulator
from simulating_noise import noise_model

In [2]:
device = LocalSimulator('braket_dm')
nm = noise_model()
n = 11 # Number of qubits on device

## Classify Noise

In [3]:
from fidelity_measurement import fidelity_classifier

device_fidelity = fidelity_classifier(n, device, nm)

single_qubit_fidelity = device_fidelity.single_qubit_fidelity()
two_qubit_fidelity = device_fidelity.two_qubit_fidelity()

## Generate Test Circuits


In [4]:
from circuits.qft import qft

test_circuit_noise = qft(n, [0]*n)
test_circuit_no_noise = qft(n, [0]*n)

## Optimize Program Logic

In [5]:
import compilers.gate_cancellation as gate_cancellation
import compilers.vertex_reindex as vertex_reindex

### Resolve identities

In [6]:
test_circuit_logic_optimized_noise = gate_cancellation.get_gate_cancelled_circuit(test_circuit_noise, n)

### Optimize Vertices

In [7]:
fully_optimized_noise = vertex_reindex.reorder_overall(test_circuit_logic_optimized_noise, n, single_qubit_fidelity, two_qubit_fidelity)

Instruction('operator': H('qubit_count': 1), 'target': QubitSet([Qubit(0)]), 'control': QubitSet([]), 'control_state': (), 'power': 1)
Qubit(0)
2
H
Instruction('operator': CPhaseShift('angle': -3.141592653589793, 'qubit_count': 2), 'target': QubitSet([Qubit(1), Qubit(0)]), 'control': QubitSet([]), 'control_state': (), 'power': 1)
Qubit(1)
[6, 2]
CPhaseShift
Instruction('operator': CPhaseShift('angle': -1.5707963267948966, 'qubit_count': 2), 'target': QubitSet([Qubit(2), Qubit(0)]), 'control': QubitSet([]), 'control_state': (), 'power': 1)
Qubit(2)
[7, 2]
CPhaseShift
Instruction('operator': CPhaseShift('angle': -0.7853981633974483, 'qubit_count': 2), 'target': QubitSet([Qubit(3), Qubit(0)]), 'control': QubitSet([]), 'control_state': (), 'power': 1)
Qubit(3)
[8, 2]
CPhaseShift
Instruction('operator': CPhaseShift('angle': -0.39269908169872414, 'qubit_count': 2), 'target': QubitSet([Qubit(4), Qubit(0)]), 'control': QubitSet([]), 'control_state': (), 'power': 1)
Qubit(4)
[4, 2]
CPhaseShift


## Run Circuits

In [9]:
test_result = device.run(nm.apply(test_circuit_noise), shots=1000).result()
no_noise_result = device.run(test_circuit_no_noise, shots=1000).result()
compiled_result = device.run(nm.apply(fully_optimized_noise), shots=1000).result()

/home/anandj/.local/lib/python3.10/site-packages/braket/default_simulator/simulator.py:324: UserWarning: You are running a noise-free circuit on the density matrix simulator. Consider running this circuit on the state vector simulator: LocalSimulator("default") for a better user experience.
  warnings.warn(


## Testing overall performance

In [14]:
import numpy as np

def conv_to_vector(measurement):

    keys_sorted = sorted(measurement.keys())
    values_sorted = [measurement[key] for key in keys_sorted]

    # Convert to numpy vector
    vector = np.array(values_sorted)

    return vector


def compare_results(noisy, real):
    ans = 0
    for key in noisy:
        if key in real:
            ans += noisy[key]*real[key]

    vec_n = conv_to_vector(noisy)
    vec_r = conv_to_vector(real)
    
    ans /= np.linalg.norm(vec_n)*np.linalg.norm(vec_r)
    
    return ans

print(compare_results(test_result.measurement_counts, no_noise_result.measurement_counts))
print(compare_results(compiled_result.measurement_counts, no_noise_result.measurement_counts))

0.3378477446767669
0.3155926779447294
